# Model combining Surface energy + Heat equation + Boundary layer

In [ ]:
# external libraries
# computation
import numpy as np
import math
from scipy.optimize import minimize, minimize_scalar

# Timing
from time import perf_counter
from tqdm import tqdm

# plotting
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# saving the model
import pickle

In [ ]:
class Combined_model:
    """Simple climatic model which combines surface energy balance, heat equation and atmospheric boundary layer."""
    
    def __init__(self, Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt):
        
        self.time = time                        # how long to run the model for
        self.dt = dt                            # time step for the model
        self.n_iters = int(time // dt)          # number of iterations

        
        self.Nx = Nx                            # Number of points in x direction
        self.Nz_atmo = Nz_atmo                  # Number of points in z direction in the atmosphere
        self.Nz_soil = Nz_soil                  # Number of points in z direction in the soil
        
        self.dz_atmo  = z_size_atmo/Nz_atmo     # Distance between z grid points in atmo
        self.dz_soil  = z_size_soil/Nz_soil     # Distance between z grid points in soil
        self.dx       = x_size/Nx               # Distance between x grid points
        
        self.time_arr = np.arange(0, time, dt)  # array of times
        
        # 3D Temperature arrays - (z,x,t)
        self.lapse_rate = -0.01
        self.T_atmo_vec = np.array([268 + self.lapse_rate * (self.dz_atmo * z) for z in range(self.Nz_atmo)])
        self.T_atmo     = np.array([[self.T_atmo_vec,]*self.Nx,]*self.n_iters).T
        
        self.T_soil = 268 * np.ones((Nz_soil, Nx, self.n_iters))
        
        # Constants        
        # Atmosphere module
        self.u = 5                 # speed of fluid
        self.atmo_K = .2           # Turbulent diffusivity
        
        # Soil module
        self.soil_K = 1.2e-6     # Conductivity

        # Surface energy balance module
        self.c_p = 1004.           # specific heat [J kg^-1 K^-1]
        self.kappa = 0.41          # Von Karman constant [-]
        self.sigma = 5.67e-8       # Stefan-Bolzmann constant
        self.L = 2.83e6            # latent heat for sublimation
        
        self.z = z                 # Height for temperature measurement - 2 m
        self.z_0 = z_0             # Surface roughness

    def _init_index_arrs(self):
        self.idx_atmo = {
            'z':   np.arange(1, self.Nz_atmo-1),
            'z_d': np.arange(0, self.Nz_atmo-2),
            'z_u': np.arange(2, self.Nz_atmo),
            }
        self.idx_soil = {
            'z':   np.arange(1, self.Nz_soil-1),
            'z_d': np.arange(0, self.Nz_soil-2),
            'z_u': np.arange(2, self.Nz_soil),
            }
        self.idx_len = {
            'x':   np.arange(1, self.Nx-1),
            'x_l': np.arange(0, self.Nx-2),
            'x_r': np.arange(2, self.Nx),
            }


    def _init_surface(self):
        """Initialises surface parameters"""
        
        self.surf_T_now = np.zeros(self.Nx)      # surface temp at a timestep
        
        # add spatial variability
        self.albedo = 0.3 * np.ones(self.Nx)     # albedo
        self.surf_f = 0.7 * np.ones(self.Nx)     # Relative humidity
        self.surf_rho = 1.1 * np.ones(self.Nx)   # Air density
        self.surf_U = 2.0 * np.ones(self.Nx)     # Wind velocity
        self.surf_z_0 = 1e-3 * np.ones(self.Nx)  # surface Roughness length
        self.surf_p = 1013 * np.ones(self.Nx)    # Pressure
        
        self.Cs_t = self.kappa**2 / (np.log(self.z/self.surf_z_0)**2)
        self.Cs_q = self.Cs_t
        
        # Incoming shortwave radiation
        # computing incoming solar radiation
        # latitude
        lat = np.radians(50)
        doy = self.time_arr/self.dt/60/24 % 365
        # solar declination
        sol_dec = np.radians(-23.44 * np.cos(2*np.pi/365*(doy+10)))
        # hour angle
        hour_ang = np.radians(self.time_arr/self.dt/60%24*15)
        # incoming solar radiation
        self.surf_G = 1366 * (np.sin(lat)*np.sin(sol_dec) + np.cos(lat)*np.cos(sol_dec)*np.cos(hour_ang))
        self.surf_G[self.surf_G < 0] = 0
        # daily amplitude - 400 W/m^2
        # annual amplitude - 250 W/m^2
        #self.surf_G = 400 * -np.cos(2 * np.pi * self.time_arr / (24 * 3600)) + \
        #              250 * -np.cos(2 * np.pi * self.time_arr / (365 * 24 * 3600))
        # negative incoming shortwave radiation set to 0
        #self.surf_G[self.surf_G<50] = 50
    
    def _init_atmosphere(self):
        """Initialises atmospheric parameters"""
        # height arr
        self.height = np.array([np.arange(0,self.dz_atmo*self.Nz_atmo,self.dz_atmo),] * self.Nx).transpose()
        # lapse rate array
        self.lapse_rate_arr = self.lapse_rate * np.ones((self.Nz_atmo, self.Nx))
        self.qsat = self._mixing_ratio(self.T_atmo[:,:,0], 1013, 270, self.height)
        # Multiply with relative humidity (80 %)
        # Init moisture array with a relative humidity of 70 % at the surface and 20 % at the top
        self.q = (self.qsat.T * np.linspace(0.7, 0.2, self.Nz_atmo)).T
        self.cov = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the covariances
        self.adv = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the advection term 
        # Dimensionless parameters
        self.c = (self.u*self.dt)/self.dx
        self.d = (self.atmo_K*self.dt)/(self.dz_atmo**2)
        
    def _sat_water_vapor(self, T):
        """Calculates the saturation water vapor pressure [Pa]"""
        Ew = 6.122 * np.exp((17.67*(T-273.16)) / (T-29.66))
        return Ew

    def _hypsometric_eqn(self, p0, Tv, z):
        """Hypsometric equation to calculate the pressure at a certain height 
        when the surface pressure is given
        p0 :: surface pressure [hPa]
        Tv :: mean virtual temperature of atmosphere [K]
        z  :: height above ground [m]
        """
        return(p0/(np.exp((9.81*z)/(287.4*Tv) )))

    def _mixing_ratio(self, theta, p0, Tv, z):
        """Calculates the mixing ratio from
        theta :: temperature [K]
        p0    :: surface pressure [hPa]
        Tv    :: mean virtual temperature of atmosphere [K]
        z     :: height [m]
        """
        return(622.97 * (self._sat_water_vapor(theta)/(self._hypsometric_eqn(p0,Tv,z)-self._sat_water_vapor(theta))))

    def surface_balance(self, T_a):
        def EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ This function calculates the energy fluxes"""
            # Correction factor for incoming longwave radiation
            eps_cs = 0.23 + 0.433 * np.power(100*(f*self._sat_water_vapor(T_a))/T_a, 1.0/8.0)

            # Calculate turbulent fluxes
            H_0 = rho * self.c_p * Cs_t * U_L * (T_0 - T_a)
            E_0 = rho * self.L*0.622/p * Cs_q * U_L * self._sat_water_vapor(T_0)*(1 - f)

            # Calculate radiation budget
            L_d = eps_cs * self.sigma * T_a**4
            L_u = 1.0 * self.sigma * T_0**4
            Q_0 = (1-albedo) * G

            return (Q_0, L_d, L_u, H_0, E_0)


        def optim_T0(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ Optimization function for surface temperature:

            Input: 
            T_0       : Surface temperature, which is optimized [K]
            T_a       : Air tempearature at height self.z
            """
                
            Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q)
    
            # Get residual for optimization
            res = np.abs(Q_0+L_d-L_u-H_0-E_0)

            # return the residuals
            return res

        # optimise temperature in each point in the x direction
        T_0 = 283. # temperature which to optimise from
        for x in range(len(self.surf_T_now)):
            surf_args = (T_a[x],         self.surf_f[x], self.albedo[x],
                         self.surf_G[self.iter_id], self.surf_p[x], self.surf_rho[x],
                         self.surf_U[x], self.Cs_t[x],   self.Cs_q[x],
                        )
                
            res = minimize(optim_T0,x0=T_0,args=surf_args,bounds=((100,400),), \
                         method='L-BFGS-B',options={'eps':1e-8})
            
            T_0 = res.x[0]

            self.surf_T_now[x] = res.x[0]
            
        return self.surf_T_now

    def boundary_layer(self, T):
        """ Simple advection-diffusion equation.
        Nz          :: Number of grid points
        dt          :: time step in seconds
        K           :: turbulent diffusivity
        u           :: Speed of fluid
        """
        # Set BC top (Neumann condition)
        # The last term accounts for the fixed gradient of 0.01
        T[-1, :] = T[-2, :] - 0.01 * self.dz_atmo #- 0.005 * self.dz_atmo

        # Set top BC for moisture
        self.q[-1, :] = self.q[-2, :] 

        # Set BC right (Dirichlet condition)
        T[:, -1] = T[:, -2]

        # Set right BC for moisture
        self.q[:, -1] = self.q[:, -2]
        
        old_t = T.copy()
        old_q = self.q.copy()

        # First update grid cells in z-direction. Here, we loop over all x grid cells and
        # use the index arrays m, mu, md to calculate the gradients for the
        # turbulent diffusion (which only depends on z)
        for x in range(self.Nx):
            # Temperature diffusion + lapse rate 
            T[self.idx_atmo['z'],x] = T[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * \
                (old_t[self.idx_atmo['z_u'],x]+old_t[self.idx_atmo['z_d'],x]-2*old_t[self.idx_atmo['z'],x]) + \
                self.lapse_rate_arr[self.idx_atmo['z'],x]
            # Turbulent diffusion of moisture
            
            self.q[self.idx_atmo['z'],x] = self.q[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * (old_q[self.idx_atmo['z_u'],x]+old_q[self.idx_atmo['z_d'],x]-2*old_q[self.idx_atmo['z'],x])
            # Calculate the warming rate [K/s] by covariance
            self.cov[self.idx_atmo['z'],x] = ((self.atmo_K)/(self.dz_atmo**2)) * (old_t[self.idx_atmo['z_u'],x] + old_t[self.idx_atmo['z_d'],x] - 2*old_t[self.idx_atmo['z'],x])
            

        # Then update grid cells in x-direction. Here, we loop over all z grid cells and
        # use the index arrays k, kl, kr to calculate the gradients for the
        # advection (which only depends on xx)
        
        for z in range(1,self.Nz_atmo-1):
            # temperature advection
            T[z,self.idx_len['x']] = T[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * \
                (old_t[z,self.idx_len['x']]-old_t[z,self.idx_len['x_l']])
            # moisture advection
            
            self.q[z,self.idx_len['x']] = self.q[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * (old_q[z,self.idx_len['x']]-old_q[z,self.idx_len['x_l']])
            
            # Calculate the warming rate [K/s] by the horizontal advection 
            # Note: Here, we use a so-called upwind-scheme (backward discretization)
            #adv[z,k] = - (u/dx)*(old[z,k]-old[z,kl])
        

        # Calculate new saturation mixing ratio
        self.qsat = self._mixing_ratio(T, 1013, 270, self.height)

        # Then the relative humidity using qsat
        rH = np.minimum(self.q/self.qsat, 1)

        # Correct lapse rates where rH==100% (moist adiabatic lapse rate)
        self.lapse_rate_arr[rH==1] = -0.006

        return T
    
    def heat_equation(self, T):
        # Set lower BC - Neumann condition
        T[0, :] = T[1, :]
            
        for x in range(self.Nx):
            # Update temperature using indices arrays
            T[self.idx_soil['z'], x] = T[self.idx_soil['z'], x] + \
                (T[self.idx_soil['z_u'], x] - 2 * T[self.idx_soil['z'], x] + \
                T[self.idx_soil['z_d'], x]) / self.dz_soil**2 * self.dt * self.soil_K

        return T
        
    def step(self):

        self.T_atmo[:,:,self.iter_id+1] = self.boundary_layer(self.T_atmo[:, :, self.iter_id])
        self.T_soil[:,:,self.iter_id+1] = self.heat_equation(self.T_soil[:, :, self.iter_id])
        
        # apply surface balance equation
        self.T_atmo[0, :, self.iter_id+1] = self.surface_balance(self.T_atmo[0, :, self.iter_id])
        
        self.T_soil[-1,:, self.iter_id+1] = self.T_atmo[0, :, self.iter_id+1]

    def run(self, verbose=False):

        a = perf_counter()
        
        self._init_index_arrs()
        self._init_surface()
        self._init_atmosphere()
        
        for idx in tqdm(range(self.n_iters-1)):
            if verbose:
                print(f'Epoch # {idx+1}/{self.n_iters}')
            self.iter_id = idx
            self.step()

        print(f'Model run finished in {perf_counter() - a} seconds.')
    
    def show_plot(self):
        concat_arr = np.concatenate((self.T_soil, 272.15*np.ones((1, self.Nx, self.n_iters)),
                                     self.T_atmo), axis=0) - 272.15
        
        fig = px.imshow(np.moveaxis(concat_arr[::-1,:,:], 2, 0), animation_frame=0,
                color_continuous_scale='RdBu_r', aspect='equal', zmin=-30, zmax=30)
        fig.show()
    
    def show_matplot(self, time_id):
        fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, gridspec_kw={'height_ratios': [4, 1]})
        img1 = ax1.imshow(self.T_atmo[:,:,time_id][::-1,:] - 272.15, vmin=-100, vmax=100)
        ax2.imshow(self.T_soil[:,:,time_id][::-1,:] - 272.15, vmin=-100, vmax=100)
        
        cbar = plt.colorbar(img1, ax=[ax1, ax2], label='Temperature [°C]')
        #cbar.set_label('Temperature [°C]')
        
    def show_matplot_time(self, x_id=0):
        fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, constrained_layout=True,
                                       gridspec_kw={'height_ratios': [4, 1]})
        img1 = ax1.imshow(self.T_atmo[:,x_id,:][::-1,:] - 272.15, aspect='auto', vmin=-150, vmax=150)
        ax2.imshow(self.T_soil[:,x_id,:][::-1,:] - 272.15, aspect='auto', vmin=-150, vmax=150)
        
        cbar = plt.colorbar(img1, ax=[ax1, ax2], label='Temperature [°C]')
        cbar.set_label('Temperature [°C]')
        
    def save_model(self, outfile):
        try:
            with open(outfile, "wb") as f:
                pickle.dump(self, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as ex:
            print("Error during pickling object (Possibly unsupported):", ex)
        

In [ ]:
def load_model(outfile):
    """Returns previously saved model."""
    try:
        with open(outfile, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

In [ ]:
# Model parameters
Nx = 2                      # Nx             number of simulated points in the x direction
Nz_atmo = 20                 # Nz_atmo        number of simulated points in the z direction (atmosphere)
Nz_soil = 4                 # Nz_soil        number of simulated points in the z direction (soil)
x_size = 1_000               # x_size         how long of a line does the model cover
z_size_atmo = 200           # z_size_atmo    how high does the model reach
z_size_soil = 2              # z_size_soil    how deep does the model reach
z = 2.0                      # z              height of tempearature measurement above the ground
z_0 = 1e-3                   # 
time = 3600 * 24 * 365 * 2   # time           how long does the model run [s]
dt = 60                      # dt             time step length [s]

In [ ]:
model = Combined_model(Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt)

In [ ]:
model.run()

In [ ]:
model.save_model('C:/Users/dd/Documents/NATUR_CUNI/HU/HU-ClimateModeling/final/results/model_overnight_test_2.pickle')

In [ ]:
model.show_matplot(-1)

In [ ]:
model.show_matplot_time()

In [ ]:
loaded_model = load_model('C:/Users/dd/Documents/NATUR_CUNI/HU/HU-ClimateModeling/final/results/model_overnight_test_2.pickle')

In [ ]:
loaded_model.show_matplot_time()

## Visualisations

In [ ]:
plt.plot(loaded_model.time_arr[int(0.5*60*24):int(7.5*60*24)]/3600/24, loaded_model.T_atmo[0,0,int(0.5*60*24):int(7.5*60*24)]-272.15)
plt.title('Surface temperature over the first week of January')
plt.xlabel('Time [days]')
plt.ylabel('Temperature [°C]')

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, constrained_layout=True,
    gridspec_kw={'height_ratios': [4, 1]}, dpi=300)
img1 = ax1.imshow(model.T_atmo[:,0,int(60*24*180.5):int(60*24*187.5)][::-1,:] - 272.15, aspect='auto', vmin=-150, vmax=150)
ax2.imshow(model.T_soil[:,0,int(60*24*180.5):int(60*24*187.5)][::-1,:] - 272.15, aspect='auto', vmin=-150, vmax=150)
#ax2.set_xticks([0,0,262800,525600, 788400, 1051199])
ax1.set_xticks([0,0,1440,2880,4320,5760,7200,8640,10080])
ax2.set_xticks([0,0,1440,2880,4320,5760,7200,8640,10080])
ax2.set_xticklabels([0,0,1,2,3,4,5,6,7])
ax1.set_yticklabels([0,200,175,150,125,100,75,50,25])
ax2.set_yticklabels([0,0,-1])
ax1.set_ylabel('Height [m]')
ax2.set_ylabel('Depth [m]')
ax2.set_xlabel('Time [days]')

cbar = plt.colorbar(img1, ax=[ax1, ax2], label='Temperature [°C]')
cbar.set_label('Temperature [°C]')

## Visualisations of sw radiation inputs

In [ ]:
time = 3600 * 24 * 365 * 3
dt = 60
time_arr = np.arange(0, time, dt)
# computing incoming solar radiation from solar azimuth angle
# latitude
lat = np.radians(52.5)
doy = time_arr/dt/60/24 % 365
# solar declination
sol_dec = np.radians(-23.44 * np.cos(2*np.pi/365*(doy+10)))
# hour angle
hour_ang = np.radians(time_arr/dt/60%24*15)
# incoming solar radiation
surf_G = 1366 * (np.sin(lat)*np.sin(sol_dec) + np.cos(lat)*np.cos(sol_dec)*np.cos(hour_ang))
surf_G[surf_G < 0] = 0

plt.figure(figsize=(9,3), dpi=300)
plt.plot(time_arr[int(60*24*0):int(60*24*365)]/3600/24, surf_G[int(60*24*0):int(60*24*365)])
plt.title('Incoming shortwave radiation at 50° latitude throughout a year')
plt.xlabel('Time [doy]')
plt.ylabel('Incoming shortwave radiation [W/m^2]')

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(8,6), nrows=2, sharey=True, constrained_layout=True, dpi=300)
ax1.plot(time_arr[int(60*24*0):int(60*24*7)]/3600/24, surf_G[int(60*24*0.5):int(60*24*7.5)])
ax2.plot(time_arr[int(60*24*180):int(60*24*187)]/3600/24, surf_G[int(60*24*180.5):int(60*24*187.5)])
fig.suptitle('Incoming shortwave radiation at 50° latitude in different parts of the year')
ax1.set_xlabel('Time [doy]')
ax1.set_ylabel('Incoming shortwave radiation [W/m^2]')
ax2.set_ylabel('Incoming shortwave radiation [W/m^2]')

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=300)
ax1 = plt.axes()
ax1.plot(time_arr[int(60*24*0):int(60*24*7)]/3600/24, surf_G[int(60*24*0.5):int(60*24*7.5)], color='blue', label='Winter')
ax2 = plt.twiny(ax1)
ax2.plot(time_arr[int(60*24*180):int(60*24*187)]/3600/24, surf_G[int(60*24*180.5):int(60*24*187.5)], color='orange', label='Summer')

ax1.set_xlabel('Time [doy]')
ax1.set_ylabel('Incoming SW radiation [W/m^2]')
ax2.set_title('Incoming shortwave radiation at 50° latitude in the winter and summer')

ax1.set_xticklabels([0,0,1,2,3,4,5,6,7], color='blue')
ax2.set_xticklabels([0,130,131,132,133,134,135,136,137], color='orange')
fig.legend(loc='center right')